In [ ]:
import pandas as pd 
import numpy as np 

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import acf, pacf, plot_acf, plot_pacf

import matplotlib.pyplot as plt 
import seaborn as sns 

sns.set()

In [ ]:
df = pd.read_csv('./ui_timeseries.csv', sep=',')

df.drop(columns=['Unnamed: 0'], inplace=True)

df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df['Periode'] = df['Periode'].astype('datetime64[ns]')

df.dtypes

In [ ]:
df['Periode'] = df['Periode'].astype('datetime64[ns]')

df.head()

In [ ]:
df = df.set_index('Periode')

df.head()

In [ ]:
adf_test = adfuller(df['Permintaan (unit)'])

In [ ]:
def test_stationarity_adf(df):
    adf_test = adfuller(df['Permintaan (unit)'])
    
    adf_df = pd.DataFrame(columns=['Statistic Test', 'p-value', 'No. of lag used', 'No. of observation', 
                               'Critical value (1%)', 'Critical value (5%)', 'Critical value (10%)'], 
                               data=[[adf_test[0], adf_test[1], adf_test[2], adf_test[3], 
                                     adf_test[4]['1%'], adf_test[4]['5%'], adf_test[4]['10%']]])
    
    return adf_df

In [ ]:
def plot_timeseries(df):
    plt.figure(figsize=(14,10))

    plt.plot(df['Permintaan (unit)'], color='blue', label="data")
    plt.plot(df['Permintaan (unit)'].rolling(7).mean(), color='black', label="mean")
    plt.plot(df['Permintaan (unit)'].rolling(7).std(), color='red', label="standar deviasi")

    plt.xticks(rotation=45)
    plt.tight_layout()

    plt.legend(loc='best')

In [ ]:
test_stationarity_adf(df)

In [ ]:
plot_timeseries(df)

In [ ]:
df = df.diff(1)

df[df['Permintaan (unit)'].isnull()]

In [ ]:
df = df.dropna()

df[df['Permintaan (unit)'].isnull()]

In [ ]:
test_stationarity_adf(df)

In [ ]:
plot_timeseries(df)

In [ ]:
plot_acf(df['Permintaan (unit)'])

In [ ]:
plot_pacf(df['Permintaan (unit)'])

In [ ]:
df.drop(columns=['Series'])

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(df, order=(3,1,1))
model.fit(df)